In [ ]:
# 2023/9/28
# zhangzhong

In [ ]:
import torch
import torch.nn.functional as F
from torch import Tensor

In [ ]:
import torch
from mytorch import training, losses, optim

# TODO: 
# 训练的效果很差呀
# 是不是哪里写错了？？
mylm = torch.load('DeepBiGRU copy')
mylm.predict(prompt='my name is ', max_len=64)

In [ ]:
a = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(a)
# https://stackoverflow.com/questions/71723788/how-to-reverse-order-of-rows-in-a-tensor
# https://pytorch.org/docs/stable/generated/torch.flip.html
b = torch.flip(a, [0])
print(b)

In [ ]:

import training
import data_module
import losses
import module
import optim
import torch

def test_language_model():
    time_machine = data_module.TimeMachineDataset(num_seq=16)
    vocab = time_machine.get_vocabulary()
    hidden_size = 64

    # https://pytorch.org/docs/stable/generated/torch.nn.RNN.html
    rnn = torch.nn.RNN(input_size=len(
        vocab), hidden_size=hidden_size, nonlinearity='relu')
    lm = module.LanguageModel(vocab=vocab, rnn=rnn, hidden_size=hidden_size)

    batch_size = 32
    num_epochs = 200
    learning_rate = 0.01
    gradient_clip = 1.0
    train_dataloader, val_dataloader = time_machine.get_dataloader(
        batch_size=batch_size)

    trainer = training.Trainer(
        model=lm,
        loss_fn=losses.CrossEntropyLoss(),
        optimizer=optim.SGD(params=lm.parameters(),
                            lr=learning_rate, gradient_clip=gradient_clip),
        num_epochs=num_epochs,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader
    )

    trainer.train(tag='RNN_LM')
    return lm
    
lm = test_language_model()

In [ ]:
result = lm.predict('introduce yourself ', max_len=64)
print(result)

In [ ]:
torch.save(lm, 'lm.pth')



In [ ]:
mylm = torch.load('lm.pth')


In [ ]:
# 卧槽 是真的可以 牛逼啊 pytorch！！
mylm.predict(prompt='my name is', max_len=64)

In [ ]:
x = torch.tensor([1, 2, 0])
F.one_hot(x, 4)

In [ ]:
x = torch.tensor([2])
F.one_hot(x, 3)

In [ ]:
# https://pytorch.org/docs/stable/generated/torch.nn.functional.one_hot.html
x = torch.tensor([[1, 2], [0, 3]])
x_one_hot = F.one_hot(x, 4)

# 其实就是之前对应位置的某一个数字 变成一个一个one_hot vector
print(x[0, 0], x_one_hot[0, 0])
print(x[0, 1], x_one_hot[0, 1])
print(x[1, 0], x_one_hot[1, 0])
print(x[1, 1], x_one_hot[1, 1])

In [ ]:
# 再尝试一下交换维度
# batch_size = 2, num_seq = 3
x = torch.tensor([[0,1,2], [3,4,5]])
x_one_hot = F.one_hot(x, 6)
print(x_one_hot)

# 现在我们想遍历所有的样本 
for x in x_one_hot:
    print(x)

In [ ]:
# 完全符合我们的推导 太对了!!
# https://pytorch.org/docs/stable/generated/torch.transpose.html
# Returns a tensor that is a transposed version of input. The given dimensions dim0 and dim1 are swapped.
x_t = torch.transpose(x_one_hot, 0, 1)
for x in x_t:
    print(x)

In [ ]:
# yield from
def funa():
    yield 1
    yield 2
    yield 3
    
def funb():
    yield from funa()
    yield 4
    yield 5
    
for x in funb():
    print(x)

In [ ]:
# list of Tensor can not convert to a whole tensor
# outputs: list[Tensor] = []
# outputs.append(torch.tensor([1,2,3]))
# outputs.append(torch.tensor([1,2,3]))
# outputs.append(torch.tensor([1,2,3]))
# outputs.append(torch.tensor([1,2,3]))
# output = torch.tensor(outputs)

# so you can use torch.stack
# https://pytorch.org/docs/stable/generated/torch.stack.html
# Concatenates a sequence of tensors along a new dimension.
# All tensors need to be of the same size.

x = torch.tensor([1,2,3])
xs = [x, x, x, x]
torch.stack(xs)

In [ ]:
x = torch.tensor([[1,2,],[3,4,]])
xs = [x, x, x, x]
torch.stack(xs)

In [ ]:
y = torch.stack(xs, dim=1)
y.shape, y

In [ ]:
# 向量各分量的平方和 就是 l2范数
# x = torch.tensor([1,2,3])
# x.norm(1)

In [ ]:
# grad clip
# 二维矩阵的grad长什么样子
x = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32, requires_grad=True)
b = torch.tensor([1, 2], dtype=torch.float32, requires_grad=True)
y = x + b
y.backward(torch.ones_like(x))
print(x.grad)


In [ ]:
import training
import data_module
import losses
import module
import optim
import torch

def test_RNN_scratch():
    
    time_machine = data_module.TimeMachineDataset(num_seq=16)
    vocab = time_machine.get_vocabulary()
    
    rnn = module.RNNScratch(vocab_size=len(vocab), num_hidden=64)
    lm = module.LanguageModelScratch(vocab=vocab, rnn=rnn)
    
    batch_size = 2048
    num_epochs = 20
    learning_rate = 0.01
    gradient_clip = 1.0
    train_dataloader, val_dataloader = time_machine.get_dataloader(batch_size=batch_size)
    
    trainer = training.Trainer(
        model=lm,
        loss_fn=losses.CrossEntropyLoss(),
        optimizer=optim.SGD(params=lm.parameters(), lr=learning_rate, gradient_clip=gradient_clip),
        num_epochs=num_epochs,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader
    )
    
    trainer.train(tag='RNN Scratch')
    return lm

In [ ]:
lm = test_RNN_scratch()

In [ ]:
msg = ''.join(lm.predict('introduce yourself ', num_seq=128))
print(msg)